In [ ]:
# Install dependencies for pulling OSM data
!pip install leafmap geopandas osmnx shapely
import leafmap
import osmnx
import geopandas
import shapely

In [ ]:
# Create a map over Parkway West
m = leafmap.Map(center=[38.622800689786146, -90.53972968259362], zoom=15)
m

In [ ]:
# Let's add some satellite imagery!
m = leafmap.Map(center=[38.622800689786146, -90.53972968259362], zoom=15)
m.add_tile_layer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    name="Google Satellite",
    attribution="Google",
)
m

In [ ]:
# Chris is running errands at home depot and gets hungry.. Let's make a new map and find some restaraunts!
area_map = leafmap.Map(center=[38.506100126287556, -90.32222702087451], zoom=17)
area_map

In [ ]:
north, south, east, west = 38.478454,38.531247,-90.361691,-90.279980
tags = {
    "building": ["yes", "house", "amenity"],
    "house": "yes"
}

# Pass bbox as a tuple: (north, south, east, west)
gdf = osmnx.features_from_bbox(bbox=(west, south, east, north), tags=tags)
gdf

In [ ]:
# Convert the projection system from geographic to projected
gdf = gdf.to_crs(epsg=3395)  
gdf

In [ ]:
# Add all buildings which met query criteria above to map
area_map.add_gdf(gdf, layer_name="Buildings", fill_colors=["green"])
area_map
# Ignore the error for now

In [ ]:
# Trim data frames to only contain two columns
small_gdf = gdf[["name", "addr:street", "amenity", "geometry"]]

# Make a new dataframe of only restaraunts
restaraunts = small_gdf.query("amenity in ['restaraunt', 'fast_food']")
restaraunts

In [ ]:
# Convert to points
restaraunts['geometry'] = restaraunts['geometry'].centroid
restaraunts

In [ ]:
# Add it to the map!
area_map.add_gdf(restaraunts, layer_name="Restaraunts", fill_colors=["green"])
area_map

In [ ]:
# Create a data frame of just Home Depot
home_depot = small_gdf[gdf['name'] == 'The Home Depot']
home_depot


In [ ]:
# Buffer the Home Depot by 1-mile (~1600m)
home_depot_centroid = home_depot['geometry']
home_depot['geometry'] = home_depot['geometry'].buffer(1600)
# Ignore the warning for now...

In [ ]:
# Add that buffer to the map!
area_map.add_gdf(home_depot, layer_name="Home Depot Buffer", fill_colors=["red"])
area_map

In [ ]:
restaraunts = restaraunts.reset_index(drop=True)
home_depot = home_depot.reset_index(drop=True)



In [ ]:
# Spatial join
restaurants_within_range = geopandas.sjoin(restaraunts, home_depot, how="inner", predicate="within")

# Convert to centroids
restaurants_within_range['geometry'] = restaurants_within_range['geometry'].centroid

# Drop columns from the right join Walmart table
columns_to_drop = [col for col in restaurants_within_range.columns if col.endswith("_right")]
restaurants_within_range = restaurants_within_range.drop(columns=columns_to_drop)

# Display the result
restaurants_within_range

In [ ]:
# Add the restaraunts within 1-mile to the map!
area_map.add_gdf(restaurants_within_range, layer_name="Restaraunt Choices", fill_colors=["green"])
area_map

In [ ]:
# Cast to EPSG:4326
restaurants_within_range_4326 = restaurants_within_range.to_crs(epsg=4326)

# Export to GeoJSON
restaurants_within_range_4326.to_file("restaurants_within_range.geojson", driver="GeoJSON")
